# Imports

In [1]:
import math ##For basic mathematical operations
import matplotlib.pyplot as plt
import dask.dataframe as dd  
from IPython import display ## Plot setup for Ipython
from matplotlib import cm ##  Colormap reference
from matplotlib import gridspec ##plot setups
from matplotlib import pyplot as plt ##plot setups
import numpy as np 
import pandas as pd
from sklearn.linear_model import LinearRegression
from tqdm import tqdm
import collections
import warnings
# from timeit import timeit - don't actually nee
# warnings.filterwarnings("ignore") - may want to ignore, if know what you are doing, but can be helpful
# warnings.filterwarnings(action = "once") - may want to show only once

In [2]:
MD = "metaData"
Indep = "trainIndependents"
Dep = "trainDependent"
fileNames = [MD, Indep, Dep]

# Reading in Data

- I was able to read in the dataframes quickly with Dask
- It actually has values in the dataframes, it's not just computing them later
- HOWEVER, it seems that not all of the elements are being read in somehow, e.g. there are only 373 row in the indep dataset being read in, so ... I need to fix something


- The way I was able to get the data to compute is by adding .head(n= NUMBER OF ROWS I WANT) to the end of the .read_csv command. I purposely queried for 100000 rows in both the Indep and Dep files to get all the rows, but it says I'm asking for too many rows, and limits the number of rows to 373 for Indep and 5965 for Dep
- This ends up computing the tasks of reading the dataframe instead of scheduling them, but it's equivalent to using the command x[0:3] for x == "cat" (i.e. it's longer than necessary, and definitely not the intended canonical way to get data into the Dask Dataframe

In [3]:
frames = []
fileLengths = [10, 100000, 100000]
for index, fileName in tqdm(enumerate(fileNames)):
    frames.append(dd.read_csv("../../../Downloads/TestData/" + fileName + ".csv", sep=",", sample = 5000000).head(n=fileLengths[index]))

0it [00:00, ?it/s]/opt/anaconda3/lib/python3.7/site-packages/dask/dataframe/core.py:5738: UserWarning: Insufficient elements for `head`. 10 elements requested, only 1 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(msg.format(n, len(r)))
/opt/anaconda3/lib/python3.7/site-packages/dask/dataframe/core.py:5738: UserWarning: Insufficient elements for `head`. 100000 elements requested, only 373 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(msg.format(n, len(r)))
2it [02:07, 63.78s/it]/opt/anaconda3/lib/python3.7/site-packages/dask/dataframe/core.py:5738: UserWarning: Insufficient elements for `head`. 100000 elements requested, only 5965 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(msg.format(n, len(r)))
3it [02:09, 43.11s/it]


It takes a short time to fill in the frames' NA data with the mean value for each column

In [4]:
timeit frames[2].fillna(frames[2].mean())

2.33 s ± 18.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
frames[2]

,Time,Symbol 0000 Dependent,Symbol 0001 Dependent,Symbol 0002 Dependent,Symbol 0003 Dependent,Symbol 0004 Dependent,Symbol 0005 Dependent,Symbol 0006 Dependent,Symbol 0007 Dependent,Symbol 0008 Dependent,...,Symbol 1144 Dependent,Symbol 1145 Dependent,Symbol 1146 Dependent,Symbol 1147 Dependent,Symbol 1148 Dependent,Symbol 1149 Dependent,Symbol 1150 Dependent,Symbol 1151 Dependent,Symbol 1152 Dependent,Symbol 1153 Dependent
0,20070103_093000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20070103_100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20070103_103000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20070103_110000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20070103_113000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5960,20081029_113000,0.013293,-0.007345,NaN,NaN,0.016008,NaN,0.000467,NaN,-0.031077,...,NaN,0.006525,NaN,NaN,0.030281,NaN,NaN,NaN,NaN,NaN
5961,20081029_120000,0.007202,-0.003475,NaN,NaN,0.008243,NaN,0.001082,NaN,-0.023354,...,NaN,0.010568,NaN,NaN,0.015491,NaN,NaN,NaN,NaN,NaN
5962,20081029_123000,0.014402,-0.003144,NaN,NaN,0.005938,NaN,0.003024,NaN,-0.019027,...,NaN,0.010846,NaN,NaN,0.008604,NaN,NaN,NaN,NaN,NaN
5963,20081029_130000,0.010380,-0.007529,NaN,NaN,0.011496,NaN,0.007678,NaN,-0.015027,...,NaN,0.021172,NaN,NaN,0.027696,NaN,NaN,NaN,NaN,NaN


We can see the labels across the Indep dataframe, and it seems that we have a "Time" column, 1154 "Symbols" (Symbols 0000 - 1153) and 20 Independent variables (Independent 00-19) for each of the symbols giving us 23081 columns. Similarly, we have a "Time" column and 1154 "Symbols" (Symbols 0000 - 1153) giving us 1155 columns.

In [23]:
allColumnsFrames = frames[1].columns.copy()
allColumnsFrames

Index(['Time', 'Symbol 0000 Independent 00', 'Symbol 0000 Independent 01',
       'Symbol 0000 Independent 02', 'Symbol 0000 Independent 03',
       'Symbol 0000 Independent 04', 'Symbol 0000 Independent 05',
       'Symbol 0000 Independent 06', 'Symbol 0000 Independent 07',
       'Symbol 0000 Independent 08',
       ...
       'Symbol 1153 Independent 10', 'Symbol 1153 Independent 11',
       'Symbol 1153 Independent 12', 'Symbol 1153 Independent 13',
       'Symbol 1153 Independent 14', 'Symbol 1153 Independent 15',
       'Symbol 1153 Independent 16', 'Symbol 1153 Independent 17',
       'Symbol 1153 Independent 18', 'Symbol 1153 Independent 19'],
      dtype='object', length=23081)

This could be because of the fact that I think I'm only reading in part of the data, but I collected stats on how many non-NA values we have:

For Indep:
- 373 rows x 23081 cols -> 13 rows have only 1 non-NA value in them ("Time")
- Remaining 360 have over 5000 non-NA value; 347 have over 6000
- Only 6341/23081 cols have at least one non-NA value!!
- No row has over 6400 non-NA values (i.e. each row is over 70% NAs)!!



For Dep:
- 5965 rows x 1155 cols
- Only 495/1155 cols have at least one non-NA value!!


In [25]:
 frames[1].transpose().dropna(how="all")

,0,1,2,3,4,5,6,7,8,9,...,363,364,365,366,367,368,369,370,371,372
Time,20070103_093000,20070103_100000,20070103_103000,20070103_110000,20070103_113000,20070103_120000,20070103_123000,20070103_130000,20070103_133000,20070103_140000,...,20070212_153000,20070213_093000,20070213_100000,20070213_103000,20070213_110000,20070213_113000,20070213_120000,20070213_123000,20070213_130000,20070213_133000
Symbol 0000 Independent 00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.000751093,0.00178328,0.00402669,0.00177286,0.000597435,0.00163539,-0.00150718,-0.000400597,0.00255282,0
Symbol 0000 Independent 01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.00237919,0.00178328,0.00402669,0.00355744,0.00461984,0.00341361,-0.000905851,0.00123614,0.00104919,-0.000400597
Symbol 0000 Independent 02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.000135696,0.00178328,0.00402669,0.00355744,0.00461984,0.0052056,0.00312475,0.00481182,0.00569581,0.00301733
Symbol 0000 Independent 03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.00421402,-0.000366864,0.00418107,0.00440583,0.00503669,0.00508127,0.00218397,0.00304952,0.00443571,0.00242923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Symbol 1148 Independent 15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
Symbol 1148 Independent 16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
Symbol 1148 Independent 17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
Symbol 1148 Independent 18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.890168,0.366502,0.71453,0.35184,0.779076,1.24694,1.02984,1.47942,1.27977,1.14566


In [8]:
frames[2].dropna(how="all")

,Time,Symbol 0000 Dependent,Symbol 0001 Dependent,Symbol 0002 Dependent,Symbol 0003 Dependent,Symbol 0004 Dependent,Symbol 0005 Dependent,Symbol 0006 Dependent,Symbol 0007 Dependent,Symbol 0008 Dependent,...,Symbol 1144 Dependent,Symbol 1145 Dependent,Symbol 1146 Dependent,Symbol 1147 Dependent,Symbol 1148 Dependent,Symbol 1149 Dependent,Symbol 1150 Dependent,Symbol 1151 Dependent,Symbol 1152 Dependent,Symbol 1153 Dependent
0,20070103_093000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20070103_100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20070103_103000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20070103_110000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20070103_113000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5960,20081029_113000,0.013293,-0.007345,NaN,NaN,0.016008,NaN,0.000467,NaN,-0.031077,...,NaN,0.006525,NaN,NaN,0.030281,NaN,NaN,NaN,NaN,NaN
5961,20081029_120000,0.007202,-0.003475,NaN,NaN,0.008243,NaN,0.001082,NaN,-0.023354,...,NaN,0.010568,NaN,NaN,0.015491,NaN,NaN,NaN,NaN,NaN
5962,20081029_123000,0.014402,-0.003144,NaN,NaN,0.005938,NaN,0.003024,NaN,-0.019027,...,NaN,0.010846,NaN,NaN,0.008604,NaN,NaN,NaN,NaN,NaN
5963,20081029_130000,0.010380,-0.007529,NaN,NaN,0.011496,NaN,0.007678,NaN,-0.015027,...,NaN,0.021172,NaN,NaN,0.027696,NaN,NaN,NaN,NaN,NaN


I can't do dropna along axis = 1 b/c of fact that Dask doesn't follow Pandas' API exactly, so I have to transpose. I filled in the mean variables, and then dropped the vars that were still NA. It works pretty quickly.

In [10]:
frames[1].fillna(frames[1].mean()).transpose().dropna().transpose()

,Time,Symbol 0000 Independent 00,Symbol 0000 Independent 01,Symbol 0000 Independent 02,Symbol 0000 Independent 03,Symbol 0000 Independent 04,Symbol 0000 Independent 05,Symbol 0000 Independent 06,Symbol 0000 Independent 07,Symbol 0000 Independent 08,...,Symbol 1148 Independent 10,Symbol 1148 Independent 11,Symbol 1148 Independent 12,Symbol 1148 Independent 13,Symbol 1148 Independent 14,Symbol 1148 Independent 15,Symbol 1148 Independent 16,Symbol 1148 Independent 17,Symbol 1148 Independent 18,Symbol 1148 Independent 19
0,20070103_093000,-0.000228384,-0.000436468,-0.000782368,-0.00184479,-0.00360093,-0.00708259,-0.0134061,-0.000242063,-0.000500845,...,4.53708e-05,-0.00194934,-0.00949958,-0.0156044,0.00106425,-0.00259916,0,0.00134693,0.88596,0.447809
1,20070103_100000,-0.000228384,-0.000436468,-0.000782368,-0.00184479,-0.00360093,-0.00708259,-0.0134061,-0.000242063,-0.000500845,...,4.53708e-05,-0.00194934,-0.00949958,-0.0156044,0.00106425,-0.00259916,0,0.00134693,0.88596,0.447809
2,20070103_103000,-0.000228384,-0.000436468,-0.000782368,-0.00184479,-0.00360093,-0.00708259,-0.0134061,-0.000242063,-0.000500845,...,4.53708e-05,-0.00194934,-0.00949958,-0.0156044,0.00106425,-0.00259916,0,0.00134693,0.88596,0.447809
3,20070103_110000,-0.000228384,-0.000436468,-0.000782368,-0.00184479,-0.00360093,-0.00708259,-0.0134061,-0.000242063,-0.000500845,...,4.53708e-05,-0.00194934,-0.00949958,-0.0156044,0.00106425,-0.00259916,0,0.00134693,0.88596,0.447809
4,20070103_113000,-0.000228384,-0.000436468,-0.000782368,-0.00184479,-0.00360093,-0.00708259,-0.0134061,-0.000242063,-0.000500845,...,4.53708e-05,-0.00194934,-0.00949958,-0.0156044,0.00106425,-0.00259916,0,0.00134693,0.88596,0.447809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,20070213_113000,0.00163539,0.00341361,0.0052056,0.00508127,-0.00202301,0.00237274,0.013091,0.0033054,0.0091612,...,0.00988115,0.022534,0.0192684,0.0290737,0,0,0,0,1.24694,0.829262
369,20070213_120000,-0.00150718,-0.000905851,0.00312475,0.00218397,-0.00369795,7.6581e-05,0.0113215,0.000524692,0.00180851,...,0.00893304,0.0225608,0.0219418,0.0281824,0,0,0,0,1.02984,0.609829
370,20070213_123000,-0.000400597,0.00123614,0.00481182,0.00304952,-0.000187167,0.00145257,0.0129083,-0.000517294,0.00275985,...,0.0101114,0.0199188,0.0249987,0.028798,0,0,0,0,1.47942,1.11276
371,20070213_130000,0.00255282,0.00104919,0.00569581,0.00443571,0.00140259,0.00243234,0.0153478,0.00459651,0.00513032,...,0.0163573,0.0222727,0.0305404,0.0337308,0,0,0,0,1.27977,1.06275


In [11]:
frames[2] #.fillna(frames[2].mean()).transpose().dropna().transpose()

,Time,Symbol 0000 Dependent,Symbol 0001 Dependent,Symbol 0002 Dependent,Symbol 0003 Dependent,Symbol 0004 Dependent,Symbol 0005 Dependent,Symbol 0006 Dependent,Symbol 0007 Dependent,Symbol 0008 Dependent,...,Symbol 1144 Dependent,Symbol 1145 Dependent,Symbol 1146 Dependent,Symbol 1147 Dependent,Symbol 1148 Dependent,Symbol 1149 Dependent,Symbol 1150 Dependent,Symbol 1151 Dependent,Symbol 1152 Dependent,Symbol 1153 Dependent
0,20070103_093000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20070103_100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20070103_103000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20070103_110000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20070103_113000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5960,20081029_113000,0.013293,-0.007345,NaN,NaN,0.016008,NaN,0.000467,NaN,-0.031077,...,NaN,0.006525,NaN,NaN,0.030281,NaN,NaN,NaN,NaN,NaN
5961,20081029_120000,0.007202,-0.003475,NaN,NaN,0.008243,NaN,0.001082,NaN,-0.023354,...,NaN,0.010568,NaN,NaN,0.015491,NaN,NaN,NaN,NaN,NaN
5962,20081029_123000,0.014402,-0.003144,NaN,NaN,0.005938,NaN,0.003024,NaN,-0.019027,...,NaN,0.010846,NaN,NaN,0.008604,NaN,NaN,NaN,NaN,NaN
5963,20081029_130000,0.010380,-0.007529,NaN,NaN,0.011496,NaN,0.007678,NaN,-0.015027,...,NaN,0.021172,NaN,NaN,0.027696,NaN,NaN,NaN,NaN,NaN


I wanted to see if there were patterns with if groups of certain labels always had values together i.e. if Symbol 1 Indep 15 and Symbol 2 Indep 17 tend to always have non-NA values together while everything else has NA values. I looked what labels had values in each row and tallied the groups of labels: 

In [12]:
allMaps = []
for i in frames:
    newMap = collections.defaultdict(int)
    for j in range(i.shape[0]):
#         print(i.loc[j].dropna(how = "all").axes[0].values)
        newMap[tuple(i.loc[j].dropna(how = "all").axes[0].values)] += 1
    allMaps.append(newMap)

In [13]:
allKeys = []
for eachMap in allMaps:
    for key, value in eachMap.items():
        allKeys.append(key)
        print(value)

1
13
13
26
13
13
13
13
26
13
13
26
13
13
13
52
39
26
13
13
9
12
13
26
13
13
13
13
26
13
13
26
13
13
13
52
39
26
26
13
39
26
52
13
26
13
39
13
13
143
13
39
78
13
13
13
26
39
91
26
26
13
65
26
39
52
26
26
13
52
13
13
13
13
26
13
26
13
39
20
13
13
39
26
26
52
13
13
13
39
13
26
52
26
117
130
26
39
52
13
13
26
26
39
13
26
13
39
13
39
26
26
13
39
13
13
13
13
26
13
13
13
13
13
13
13
13
33
13
13
13
39
65
39
39
13
59
26
13
39
13
13
13
13
13
13
26
13
13
26
39
13
13
13
117
104
39
39
13
13
13
26
13
26
13
39
13
13
26
26
65
13
13
13
13
13
52
13
13
26
26
26
13
13
13
13
13
26
26
26
13
13
39
13
52
13
26
26
39
13
13
39
13
13
26
13
26
7
26
13
13
13
13
13
13
13
13
13
13
26
13
26
13
13
13
13
13
13
13
13
13
13
13
26
39
13
13
13
13
26
26
13
26
13
13
13
13
26
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
26
13
13
13
13
13
13
13
13
13
23


The above thing showed me there were certain correlations in both Indep and Dep, but not which variables tended to be correlated with regards to when they had values. More importantly, I wanted to know if all the Independent variables corresponding to a specific Symbol tended to exhibit the same behavior - e.g. all be NA or non-NA together at once. This was in fact the case for each timestep - if one independent variables for one Symbol is non-NA, so are the rest, and if it is NA, so are the rest.

Also, the symbols whose Indep variables were showing up tended to occur about the same # of times: the indep variables for each symbol had non-NA values 299-316 times.

For Dep, the same thing was true: each Symbol had non-NA values 299-400 times.

In [14]:
allNewMaps = []
for i in allKeys:
    allNewMap = collections.defaultdict(int)
    for j in i:
#         print(i.loc[j].dropna(how = "all").axes[0].values)
        if len(j) > 11:
            symbolName = j[7:11]
            allNewMap[symbolName] += 1
    allNewMaps.append(allNewMap)

In [16]:
for eachMap in allNewMaps:
#     string = ""
    indepVars = 0
    depVars = 0
    for key, value in eachMap.items():
        if value == 20:
#             string += (key + " ") 
            indepVars += 1
        else:
            depVars += 1
#         if string:
    #         print(len(string)/5)
    print(indepVars, depVars)

0 1
0 0
299 0
301 0
301 0
302 0
304 0
305 0
306 0
307 0
308 0
309 0
311 0
311 0
310 0
312 0
313 0
313 0
314 0
315 0
316 0
0 0
0 299
0 301
0 301
0 302
0 304
0 305
0 306
0 307
0 308
0 309
0 311
0 311
0 310
0 312
0 313
0 313
0 314
0 315
0 313
0 314
0 315
0 316
0 317
0 319
0 320
0 321
0 322
0 324
0 325
0 324
0 325
0 326
0 328
0 329
0 330
0 331
0 330
0 331
0 332
0 333
0 333
0 334
0 335
0 334
0 334
0 333
0 335
0 335
0 336
0 337
0 338
0 339
0 339
0 340
0 341
0 340
0 341
0 342
0 341
0 342
0 343
0 344
0 345
0 346
0 347
0 347
0 346
0 347
0 348
0 347
0 349
0 350
0 351
0 352
0 351
0 352
0 352
0 353
0 352
0 351
0 350
0 351
0 352
0 351
0 352
0 351
0 353
0 351
0 352
0 353
0 354
0 355
0 356
0 357
0 358
0 359
0 361
0 363
0 364
0 365
0 364
0 364
0 362
0 361
0 362
0 363
0 364
0 365
0 366
0 368
0 369
0 368
0 368
0 369
0 368
0 367
0 369
0 369
0 370
0 370
0 371
0 371
0 371
0 372
0 373
0 372
0 372
0 373
0 372
0 371
0 371
0 370
0 371
0 372
0 373
0 374
0 375
0 374
0 375
0 376
0 375
0 376
0 375
0 376
0 376
0 37

Below, it seems to prove that the dataframe I read in is smaller than the actual data we were provided.

In [15]:
frames[1].memory_usage() # 68 900 688, 72 bytes for time, index = 128 bytes total?

Index                          128
Time                          2984
Symbol 0000 Independent 00    2984
Symbol 0000 Independent 01    2984
Symbol 0000 Independent 02    2984
                              ... 
Symbol 1153 Independent 15    2984
Symbol 1153 Independent 16    2984
Symbol 1153 Independent 17    2984
Symbol 1153 Independent 18    2984
Symbol 1153 Independent 19    2984
Length: 23082, dtype: int64

This could just be b/c I haven't read in the complete dataset, but since I have fewer rows in Indep than Dep, some time steps in Dep have no corresponding rows in Indep (i.e. that share the same "Time" value), but for every row in Indep, there does exist a corresponding row in Dep with the same "Time" value. I haven't analyzed what the relation is between rows that show up in Indep and rows that don't (i.e. do they share the same date, but a different time?) b/c I think I am working with an incomplete dataset, b/c Dask isn't reading in the full dataset.

In [17]:
allSets = []
for i in frames:
    newSet = set()
    for j in range(i.shape[0]):
    #         print(i.loc[j].dropna(how = "all").axes[0].values)
        newSet.add(i.loc[j].dropna(how = "all")[0])
    allSets.append(newSet)
allSets

[{1154},
 {'20070103_093000',
  '20070103_100000',
  '20070103_103000',
  '20070103_110000',
  '20070103_113000',
  '20070103_120000',
  '20070103_123000',
  '20070103_130000',
  '20070103_133000',
  '20070103_140000',
  '20070103_143000',
  '20070103_150000',
  '20070103_153000',
  '20070104_093000',
  '20070104_100000',
  '20070104_103000',
  '20070104_110000',
  '20070104_113000',
  '20070104_120000',
  '20070104_123000',
  '20070104_130000',
  '20070104_133000',
  '20070104_140000',
  '20070104_143000',
  '20070104_150000',
  '20070104_153000',
  '20070105_093000',
  '20070105_100000',
  '20070105_103000',
  '20070105_110000',
  '20070105_113000',
  '20070105_120000',
  '20070105_123000',
  '20070105_130000',
  '20070105_133000',
  '20070105_140000',
  '20070105_143000',
  '20070105_150000',
  '20070105_153000',
  '20070108_093000',
  '20070108_100000',
  '20070108_103000',
  '20070108_110000',
  '20070108_113000',
  '20070108_120000',
  '20070108_123000',
  '20070108_130000',
  '2

In [18]:
print(len(allSets[1]))
print(len(allSets[2]))

373
5965


This will show that since combining both sets, we still have the same number of elements (unique time values) as the dependent dataset does by itself, all the unique time values in the independent dataset are contained with the dependent datset.

In [26]:
newnewSet = allSets[1].copy()
newnewSet.update(allSets[2].copy())
print(len(newnewSet))

5965


I want to see if there is a relation between the symbols for which we have values in the independent and dependent dataset for the rows that share the time label i.e. if the time label = 20001100_000001 and the independent dataset has values for Symbol 0000 (Indep Vars 00-19), then does the dependent dataset as well? 

Yes, for the most case, as we'll see below - every symbol for which we have values (for their 20 independent variables) in the independent dataset for a specific timestep also has a value in the dependent dataset for that specific timestep for that symbol. There are exceptions, though - there are 26 timesteps for which there is more one symbol that has a value in the independent dataset then the dependent dataset.

In [20]:
allMaps = []
for i in frames:
    newMap = {}
    for j in range(i.shape[0]):
#         print(i.loc[j].dropna(how = "all").axes[0].values)
        newTuple = tuple(i.loc[j].dropna(how = "all").axes[0].values)
        newMap[i.loc[j][0]] = newTuple[1:]
    allMaps.append(newMap)

In [21]:
allMaps;

In [22]:
j = 0
for i in allMaps[1]:
    indepVars = allMaps[1][i]
    depVars = allMaps[2][i]
    indepSet = set()
    depSet = set()
    for eachVar in indepVars:
        indepSet.add(eachVar[7:11])
    for eachVar in depVars:
        depSet.add(eachVar[7:11])
    allSet = indepSet
    allSet.update(depSet)
    if indepSet != depSet: #len(indepSet), len(depSet), 
        j += 1
        print(indepSet - depSet)
        print(len(indepSet), len(depSet), j)
#     print(, " ", )
#     j += 1
#     if j > 20:
#         break
#     for j in range(i.shape[0]):

{'1081'}
302 301 1
{'0043'}
312 311 2
{'1133'}
311 310 3
{'1133'}
314 313 4
{'1133'}
315 314 5
{'1133'}
315 314 6
{'1133'}
315 314 7
{'1133'}
315 314 8
{'1133'}
315 314 9
{'1133'}
315 314 10
{'1133'}
315 314 11
{'1133'}
315 314 12
{'1133'}
315 314 13
{'1133'}
315 314 14
{'1133'}
315 314 15
{'1133'}
315 314 16
{'1133'}
316 315 17
{'1133'}
316 315 18
{'1133'}
316 315 19
{'1133'}
316 315 20
{'1133'}
316 315 21
{'1133'}
316 315 22
{'1133'}
316 315 23
{'1133'}
316 315 24
{'1133'}
316 315 25
{'1133'}
316 315 26
